# 🎯 RAG Demo: Retrieval-Augmented Generation

This notebook demonstrates a complete RAG (Retrieval-Augmented Generation) pipeline that combines:
- **Internal Knowledge**: ChromaDB with product catalog
- **External Knowledge**: Web search via SerpAPI Bing
- **AI Generation**: Azure OpenAI GPT-4
- **Quality Evaluation**: Automated response accuracy assessment

## 🏗️ Component Overview

The RAG system intelligently combines internal company data with external web information to provide comprehensive, accurate responses.

```mermaid
graph LR
    A[User Query] --> B[ChromaDB Search]
    B --> C[Generate Web Queries]
    A --> C[Generate Web Queries]
    C --> D[SerpAPI Bing Search]
    B --> E[Combine Context]
    D --> E
    E --> F[RAG: 
    Query + Context 
    -> LLM Response]
    F --> G[LLM Evaluation]
```

In [11]:
import sys
import os
import asyncio
import time
from pathlib import Path

# Add src directory to path
project_root = os.path.join(str(Path.cwd().parent), 'src')
sys.path.insert(0, project_root)

# Import RAG components
from utils.mcp_config import Config
from tools.chroma_search import ChromaDBSearcher
from tools.web_search import WebSearcher
from tools.rag_generator import RAGResponseGenerator

# Initialize configuration
config = Config(environment="local")
print(f"✅ Configuration loaded: {config.environment}")

2025-08-03 14:33:42,746 - root - INFO - ✅ Loaded environment file for 'local': c:\Users\aprilhazel\Source\sk_mcp_demo\mcp_rag\.env.local


✅ Configuration loaded: local


## 🧪 Demo Queries

We'll test the RAG system with two different scenarios:
1. **External Knowledge Needed**: "What is the best footwear for hiking?" - Requires web search
2. **Internal Knowledge Sufficient**: "Tell me about our RainGuard Hiking Jacket" - Uses product catalog

In [2]:
# Initialize RAG components
chroma_searcher = ChromaDBSearcher(config)
web_searcher = WebSearcher(config)
rag_generator = RAGResponseGenerator(config)

# Define test queries
test_queries = [
    "What is the best footwear for hiking?",  # External knowledge needed
    "Tell me about our RainGuard Hiking Jacket product."  # Internal knowledge sufficient
]

print("🎯 Demo Queries:")
for i, query in enumerate(test_queries, 1):
    print(f"  {i}. {query}")
    
print(f"\n🔧 RAG Components initialized successfully!")

🎯 Demo Queries:
  1. What is the best footwear for hiking?
  2. Tell me about our RainGuard Hiking Jacket product.

🔧 RAG Components initialized successfully!


# Scenario Discussion

- **Scenario 1** involves the user asking a generic question that `goes beyond the scope of the internal data` - a product catalog. In this scenario, external Web context may provide value; therefore relevant Web queries are generated to be executed and included in the context to answer the users question.

- **Scenario 2** involves the user asking a question about a `specific product in the internal catalog`, the RainGuard Hiking Jacket. In this scenario, the system has been instructed to only answer questions from the internal product catalog about any internal products. No Web queries are generated. The RAG response will be limited to internal data sources.

## 🔍 Internal Search - Chroma DB - Local File in MCP Server

The below shows how each scenario performs when searching the local Chroma database. The Chroma database is local to the MCP server.

In [ ]:
# Demo 1: ChromaDB Internal Search
collection_info = await chroma_searcher.get_collection_info()
print(f"🔍 Searching INTERNAL DATA (ChromaDB). {collection_info['document_count']} documents ingested from https://github.com/Azure-Samples/contoso-chat/blob/main/data/product_info/products.csv)")

internal_context0 = await chroma_searcher.search_chroma(
    query=test_queries[0],
    n_results=2
)
internal_context1 = await chroma_searcher.search_chroma(
    query=test_queries[1],
    n_results=2
)

print("="*100)
print(f"{'='*20} INTERNAL + EXTERNAL {test_queries[0]} {'='*20}")
print("="*100)

for i, result in enumerate(internal_context0[:3], 1):
    print(f"\n  {i}. Content: {result['content'][:100]}...")
    print(f"     Citation: {result['citation']}")
    print(f"     Similarity: {result['metadata'].get('similarity_score', 'N/A')}")

print("="*100)
print(f"{'='*20} INTERNAL ONLY {test_queries[1]} {'='*20}")
print("="*100)

for i, result in enumerate(internal_context1[:3], 1):
    print(f"\n  {i}. Content: {result['content'][:100]}...")
    print(f"     Citation: {result['citation']}")
    print(f"     Similarity: {result['metadata'].get('similarity_score', 'N/A')}")

🔍 Searching INTERNAL DATA (ChromaDB). 20 documents ingested from https://github.com/Azure-Samples/contoso-chat/blob/main/data/product_info/products.csv)
==================== INTERNAL + EXTERNAL What is the best footwear for hiking? ====================

  1. Content: ID: 11, Name: TrailWalker Hiking Shoes, Price: $110.0, Category: Hiking Footwear, Brand: TrekReady, ...
     Citation: [Source: ChromaDB | Collection: product_collection | ID: 11 | Similarity: 0.180]
     Similarity: 0.18037372827529907

  2. Content: ID: 4, Name: TrekReady Hiking Boots, Price: $140.0, Category: Hiking Footwear, Brand: TrekReady, Des...
     Citation: [Source: ChromaDB | Collection: product_collection | ID: 4 | Similarity: 0.117]
     Similarity: 0.11699265241622925
==================== INTERNAL ONLY Tell me about our RainGuard Hiking Jacket product. ====================

  1. Content: ID: 17, Name: RainGuard Hiking Jacket, Price: $110.0, Category: Hiking Clothing, Brand: MountainStyl...
     Citation: [So

In [6]:
internal_context0

[{'query': 'What is the best footwear for hiking?',
  'content': "ID: 11, Name: TrailWalker Hiking Shoes, Price: $110.0, Category: Hiking Footwear, Brand: TrekReady, Description: Meet the TrekReady TrailWalker Hiking Shoes, the ideal companion for all your outdoor adventures. Constructed with synthetic leather and breathable mesh, these shoes are tough as nails yet surprisingly airy. Their cushioned insoles offer fabulous comfort for long hikes, while the supportive midsoles and traction outsoles with multidirectional lugs ensure stability and excellent grip. A quick-lace system, padded collar and tongue, and reflective accents make these shoes a dream to wear. From combating rough terrain with the reinforced toe cap and heel, to keeping off trail debris with the protective mudguard, the TrailWalker Hiking Shoes have you covered. These waterproof warriors are made to endure all weather conditions. But they're not just about being rugged, they're light as a feather too, minimizing fatig

## 🌐 External Search - Bing via Serp API

Execute the below block to see how each scenario is treated for external searching. 

- Scenario 1 has web search queries generated

- Scenario 2 that is specific to an internal product does not have queries generated (this is by instruction to an LLM call that generates web queries based on the user's question and internal context provided)

In [ ]:
# Demo 2: Intelligent Web Search
print("🌐 Searching EXTERNAL DATA (Bing) with Intelligent Web Query Generation...")

generated_queries0 = await web_searcher._get_web_search_queries(
    user_query=test_queries[0],
    internal_context=internal_context0  # Remove .to_list() - already a list
)
generated_queries1 = await web_searcher._get_web_search_queries(
    user_query=test_queries[1],
    internal_context=internal_context1  # Remove .to_list() - already a list
)

print("="*100)
print(f"{'='*20} INTERNAL + EXTERNAL {test_queries[0]} {'='*20}")
print("="*100)
print(f"Generated {len(generated_queries0)} web queries:")  # generated_queries0 is a list
for query in generated_queries0:  # Iterate directly over the list
    print(f"  Priority {query['priority_rank']}: {query['search_query']}")
    print(f"  Purpose: {query['purpose']}\n")
print("="*20)
# Execute web search if queries were generated
print(f"Executing generated queries asynchronously: {len(generated_queries0)} queries")
if generated_queries0:  # Check if list is not empty
    print("🔍 Executing Web Search...")
    web_results = await web_searcher.search_serpapi_bing_with_generated_queries(
        generated_queries=generated_queries0,
        n_results=2
    )
    print(f"Returned web results (external context): {len(web_results)} results:")
    for i, result in enumerate(web_results, 1):  # web_results is a list, not SearchResults object
        print(f"\n  {i}. Content: {result['content'][:100]}...")
        print(f"     Citation: {result['citation']}")

print("="*100)
print(f"{'='*20} INTERNAL ONLY {test_queries[1]} {'='*20}")
print("="*100)
print(f"Generated {len(generated_queries1)} web queries:")  # generated_queries1 is a list
for query in generated_queries1:  # Iterate directly over the list
    print(f"  Priority {query['priority_rank']}: {query['search_query']}")
    print(f"  Purpose: {query['purpose']}\n")
print("⏭️ !!!! No search queries should be generated - internal context is sufficient !!!")
print("See the prompt in the web_search module for instructions that enforce this.")

🌐 Searching EXTERNAL DATA (Bing) with Intelligent Web Query Generation...
==================== INTERNAL + EXTERNAL What is the best footwear for hiking? ====================
Generated 2 web queries:
  Priority 1: best hiking footwear for various terrains 2025
  Purpose: To identify top-rated hiking footwear options for different terrains and conditions.

  Priority 2: comparison of hiking shoes vs hiking boots benefits
  Purpose: To understand the advantages and disadvantages of hiking shoes versus boots for hiking.

Executing generated queries asynchronously: 2 queries
🔍 Executing Web Search...
Returned web results (external context): 2 results:

  1. Content: To find the absolute best hiking shoes—lightweight backpacking footwear with rock‑solid grip, waterp...
     Citation: [Source: Bing Search | Link: https://outdoortrekker.com/footwear/hiking-footwear/best-hiking-shoes/ | Position: 1]

  2. Content: Today, a wide variety of hiking boots and hiking shoes share …...
     Citation: 

## 🤖 Explore the RAG Response

- Execute the below cell, and then, explore each RAG response as you like. 
- Consider how applications might display the response and also the citations

**🚨 Important:** The below will not use the above found context. Instead, it will perform the entire RAG response function end to end and may find new context.

In [8]:
print("🤖 Generating an LLM response to the test user queries ...\n")

print("="*100)
print(f"{'='*20} INTERNAL + EXTERNAL {test_queries[0]} {'='*20}")
print("="*100)
rag_response0 = await rag_generator.generate_chat_response(
            user_query=test_queries[0],
            n_chroma_results=3,
            n_web_results=3
        )
print("🤖 Generated RAG response, use one of the below cells to evaluate the `rag_response0` varible")
print(rag_response0["response"])

print("="*100)
print(f"{'='*20} INTERNAL ONLY {test_queries[1]} {'='*20}")
print("="*100)
rag_response1 = await rag_generator.generate_chat_response(
            user_query=test_queries[1],
            n_chroma_results=3,
            n_web_results=3
        )
print("🤖 Generated RAG response, use one of the below cells to evaluate the `rag_response1` varible")
print(rag_response1["response"])

🤖 Generating an LLM response to the test user queries ...

==================== INTERNAL + EXTERNAL What is the best footwear for hiking? ====================


2025-08-03 14:31:36,226 - src.tools.web_search - WARNING - Search timeout for query 'comparison of hiking shoes, boots, and sandals for...'


🤖 Generated RAG response, use one of the below cells to evaluate the `rag_response0` varible
The best footwear for hiking depends on the type of terrain, weather conditions, and personal preferences. Here’s a breakdown of options based on internal knowledge and external sources:

### **Internal Knowledge Base Recommendations**
1. **TrailWalker Hiking Shoes ($110.0)**:
   - **Features**: Lightweight, waterproof, breathable mesh, synthetic leather, cushioned insoles, multidirectional traction lugs, reinforced toe cap, and protective mudguard.
   - **Best For**: Long hikes on varied terrain, especially where lightweight and waterproof features are essential.
   - **Advantages**: Comfortable for extended use, customizable fit with removable insoles, and reflective accents for visibility.
   - **Brand**: TrekReady.

2. **TrekReady Hiking Boots ($140.0)**:
   - **Features**: Leather construction, reinforced stitching, toe protection, moisture-wicking lining, shock-absorbing midsoles, and exc

Explore rag_response0 - INTERNAL + EXTERNAL What is the best footwear for hiking?

In [9]:
rag_response0

{'user_query': 'What is the best footwear for hiking?',
 'response': "The best footwear for hiking depends on the type of terrain, weather conditions, and personal preferences. Here’s a breakdown of options based on internal knowledge and external sources:\n\n### **Internal Knowledge Base Recommendations**\n1. **TrailWalker Hiking Shoes ($110.0)**:\n   - **Features**: Lightweight, waterproof, breathable mesh, synthetic leather, cushioned insoles, multidirectional traction lugs, reinforced toe cap, and protective mudguard.\n   - **Best For**: Long hikes on varied terrain, especially where lightweight and waterproof features are essential.\n   - **Advantages**: Comfortable for extended use, customizable fit with removable insoles, and reflective accents for visibility.\n   - **Brand**: TrekReady.\n\n2. **TrekReady Hiking Boots ($140.0)**:\n   - **Features**: Leather construction, reinforced stitching, toe protection, moisture-wicking lining, shock-absorbing midsoles, and excellent tracti

Explore rag_response1 - INTERNAL ONLY Tell me about our RainGuard Hiking Jacket product.

In [10]:
rag_response1

{'user_query': 'Tell me about our RainGuard Hiking Jacket product.',
 'response': "The **RainGuard Hiking Jacket** is a versatile and high-performance outdoor jacket designed by **MountainStyle**, perfect for hiking, camping, trekking, and other outdoor adventures. Here are the key features of the product:\n\n### Features and Benefits:\n1. **Weatherproof Design**:\n   - Made with waterproof and breathable fabric to keep you dry and comfortable in wet conditions.\n   - Adjustable hood for a customizable fit against wind and rain.\n\n2. **Comfort and Durability**:\n   - Rugged construction ensures long-lasting durability.\n   - Adjustable cuffs and hem allow you to tailor the fit to your needs.\n\n3. **Convenience**:\n   - Multiple pockets provide safe and convenient storage for your essentials.\n   - Ventilation zippers are included to prevent overheating during intense activities.\n\n4. **Safety and Visibility**:\n   - Reflective details enhance visibility during low-light conditions, 

## 👩‍🔬 Add Evaluations Logic

- To right-size user expectations about accuracy, consider returning an accuracy score with results
- Consider if every response should evaluated for accuracy; or, should users opt-in via the MCP dynamic tools?

**🚨 Important:** The evaluation logic embedded in this solution is for demonstration purposes only. It is recommended to use a content moderation service provider, such as Azure AI Content Safety, that specializes in evaluations

In [19]:
print("👩‍🔬 Generating RAG responses with integrated evaluation ...\n")

print("="*100)
print(f"{'='*20} INTERNAL + EXTERNAL {test_queries[0]} {'='*20}")
print("="*100)

try: 
    start = time.time()
    rag_with_evaluation0 = await rag_generator.generate_evaluated_chat_response(
        user_query=test_queries[0],
        n_chroma_results=3,
        n_web_results=3,
        collection_name="product_collection"
    )
    elapsed = time.time() - start
    print(f"👩‍🔬✅ RAG response with evaluation succeeded: evaluation score: {rag_with_evaluation0['evaluation']['accuracy_score']:.2f} (Time: {elapsed:.2f}s)")
    print(f"🤖 Generated RAG response (first 100 characters): {rag_with_evaluation0['response'][:100]}...")  # Print first 100 chars of response
    print(f"\n📈 Evaluation Results:")
    evaluation0 = rag_with_evaluation0['evaluation']
    print(f"   Accuracy Score: {evaluation0['accuracy_score']:.2f}")
    print(f"   Confidence: {evaluation0['confidence_level']}")
    print(f"   Supported Claims: {len(evaluation0['supported_claims'])}")
    print(f"   Unsupported Claims: {len(evaluation0['unsupported_claims'])}")
    
except Exception as e:
    print(f"👩‍🔬❌ Error in RAG response with evaluation test: {e}")

print("="*100)
print(f"{'='*20} INTERNAL ONLY {test_queries[1]} {'='*20}")
print("="*100)

try: 
    start = time.time()
    rag_with_evaluation1 = await rag_generator.generate_evaluated_chat_response(
        user_query=test_queries[1],
        n_chroma_results=3,
        n_web_results=3,
        collection_name="product_collection"
    )
    elapsed = time.time() - start
    print(f"👩‍🔬✅ RAG response with evaluation succeeded: evaluation score: {rag_with_evaluation1['evaluation']['accuracy_score']:.2f} (Time: {elapsed:.2f}s)")
    print(f"🤖 Generated RAG response (first 100 characters): {rag_with_evaluation1['response'][:100]}...")  # Print first 100 chars of response
    print(f"\n📈 Evaluation Results:")
    evaluation1 = rag_with_evaluation1['evaluation']
    print(f"   Accuracy Score: {evaluation1['accuracy_score']:.2f}")
    print(f"   Confidence: {evaluation1['confidence_level']}")
    print(f"   Supported Claims: {len(evaluation1['supported_claims'])}")
    print(f"   Unsupported Claims: {len(evaluation1['unsupported_claims'])}")
    
except Exception as e:
    print(f"👩‍🔬❌ Error in RAG response with evaluation test: {e}")

👩‍🔬 Generating RAG responses with integrated evaluation ...

==================== INTERNAL + EXTERNAL What is the best footwear for hiking? ====================


2025-08-03 14:41:44,221 - src.tools.web_search - WARNING - Search timeout for query 'top-rated hiking shoes, boots, and sandals in 2025...'


👩‍🔬✅ RAG response with evaluation succeeded: evaluation score: 0.85 (Time: 20.94s)
🤖 Generated RAG response (first 100 characters): The best footwear for hiking depends on your specific needs, preferences, and the type of terrain yo...

📈 Evaluation Results:
   Accuracy Score: 0.85
   Confidence: High
   Supported Claims: 4
   Unsupported Claims: 2
==================== INTERNAL ONLY Tell me about our RainGuard Hiking Jacket product. ====================
👩‍🔬✅ RAG response with evaluation succeeded: evaluation score: 1.00 (Time: 8.34s)
🤖 Generated RAG response (first 100 characters): The **MountainStyle RainGuard Hiking Jacket** is a versatile and high-performance piece of hiking cl...

📈 Evaluation Results:
   Accuracy Score: 1.00
   Confidence: High
   Supported Claims: 12
   Unsupported Claims: 0


In [20]:
rag_with_evaluation0

{'user_query': 'What is the best footwear for hiking?',
 'response': "The best footwear for hiking depends on your specific needs, preferences, and the type of terrain you plan to tackle. Below, I’ll outline some options based on internal knowledge and external sources:\n\n---\n\n### **Internal Knowledge Recommendations**\n1. **TrailWalker Hiking Shoes ($110.00)**  \n   - **Best for:** Versatile hiking across various terrains.  \n   - **Features:** Lightweight, waterproof, and breathable with synthetic leather and mesh construction. They offer excellent grip with multidirectional lugs, cushioned insoles for comfort, and a quick-lace system for convenience. They are ideal for long hikes and rough terrain due to their reinforced toe cap, heel, and protective mudguard.  \n   - **Why Choose:** These shoes balance durability, comfort, and performance, making them a great all-around choice for hikers.  \n\n2. **TrekReady Hiking Boots ($140.00)**  \n   - **Best for:** Rugged terrain and chall

## 🎮 Interactive Demo

Try your own queries with the RAG system!

In [24]:
my_question = ""

if my_question == "":
    print("❓ Please enter your test question above")

else: 
      print(f"🎮 Testing the end to end flow with my_question")
      print("="*100)
      print(f"{'='*20} My question: {my_question} {'='*20}")
      print("="*100)

      my_test_response = await rag_generator.generate_evaluated_chat_response(
        user_query=my_question,
        n_chroma_results=10,
        n_web_results=10
    )

      # And, now, return the results
      print(f"""The LLM''s response to my question: 
{my_test_response['response']}""")
      print(f"""!!!!! The accuracy score of the LLM's response: {my_test_response['evaluation']['accuracy_score']:.2f} and the confidence level is {my_test_response['evaluation']['confidence_level']}. """)

❓ Please enter your test question above


Or, look at the entire contents of the output

In [29]:
my_test_response